In [14]:
import json


'''
------------------------------------------------
---  A note about comparison techniques used ---
------------------------------------------------
json_delta is best for serializing/deserializing structures and
minimizing comm overhead.  It's may not be ideal for specialized
comparison of existing JSON
'''

"\n------------------------------------------------\n---  A note about comparison techniques used ---\n------------------------------------------------\njson_delta is best for serializing/deserializing structures and\nminimizing comm overhead.  It's may not be ideal for specialized\ncomparison of existing JSON\n"

In [15]:
def parse_date(file_name):
    
    starting_point_of_date = "_20"
    date_pos_start = file_name.find(starting_point_of_date)+1
    return file_name[date_pos_start:date_pos_start+10]



'''
#--- Test it ---
parse_date("snapshots/HealthData.gov_2014-02-24_data.json")
'''



'\n#--- Test it ---\nparse_date("snapshots/HealthData.gov_2014-02-24_data.json")\n'

In [35]:
'''
------------------------------------------------
---  Capturing agency counts                 ---
------------------------------------------------
Many dataset entries lack bureauCode.  
So perhaps other identifiers can be used as a proxy
'''


# Pull out the most important elements to tally on
def get_keys(dataset):
    keys = ["bureauCode", "publisher", "landingPage","modified"]
    key_values = []
    for i,key in enumerate(keys):
        if key in dataset:
            key_values.append(dataset[key])
        else:
            key_values.append(None)
    return dict(zip(keys, key_values))
        

'''
#--- Test it ---
print get_keys(json_data_list[0][0])
'''
'''
# --- Experiment with keys ---
if "bureauCode"  in json_data_list[0][0]: print json_data_list[0][0]["bureauCode"]
if "publisher"   in json_data_list[0][0]: print json_data_list[0][0]["publisher"]
if "landingPage" in json_data_list[0][0]: print json_data_list[0][0]["landingPage"]
print json.dumps(json_data_list[0][0], sort_keys=False, indent=4)
'''

'\n# --- Experiment with keys ---\nif "bureauCode"  in json_data_list[0][0]: print json_data_list[0][0]["bureauCode"]\nif "publisher"   in json_data_list[0][0]: print json_data_list[0][0]["publisher"]\nif "landingPage" in json_data_list[0][0]: print json_data_list[0][0]["landingPage"]\nprint json.dumps(json_data_list[0][0], sort_keys=False, indent=4)\n'

In [136]:


# FIXME: Code not yet finished
# FIXME: Should call get_keys
# Create a dictionary of values for comparison

def get_key_list(dataset_list):
    key_list = []
    for index, dataset in enumerate(dataset_list):
        key_list.append(get_keys(dataset))
    #for # List of unique bureauCode values    
    
    totals = len(dataset_list)
    #print get_keys(dataset[0])
    return key_list




In [144]:
def get_agency_counts(key_list):
    agency_counts = {}
    for index,key_item in enumerate(key_list):

        agencies = key_item["bureauCode"]

        # Just in case it's not a list, make it one
        agencies = agencies if isinstance(agencies,list) else [agencies]

        for agency in agencies:
            agency = agency.encode('ascii','ignore')
            agency_counts[agency] = agency_counts.get(agency, 0) + 1
            
    return agency_counts



{'009:25': 44, '009:15': 19, '009:92': 7, '009:10': 151, '009:75': 6, '009:20': 243, '009:30': 11, '009:17': 1, '009:70': 81, '009:00': 679, '009:38': 551, '009:33': 14}


In [29]:
def support_old_schema(dataset_list):
    if isinstance(dataset_list, dict):
        return dataset_list["dataset"]
    elif isinstance(dataset, list):
        return dataset_list
    else:
        return None

'''    
#--- Test it ---
print support_old_schema(dataset)

'''

'    \n#--- Test it ---\nprint support_old_schema(dataset)\n\n'

In [153]:
import os
import glob   # Wildcard search
import json

def load_file(file_name):
    with open(file_name) as json_file:
        json_data = json.load(json_file)
        return json_data
        print("Loaded file: "+file_name)


#FIXME: Need to find agency decode
#       Examples: {'009:25', '009:15', '009:92', '009:10', '009:75',
#                  '009:20', '009:30', '009:17', '009:70', '009:00', 
#                  '009:38', '009:33'}
def main():
    file_pattern = "snapshots/"
    file_pattern += "HealthData.gov[_][0-9][0-9][0-9][0-9][-][0-9][0-9][-][0-9][0-9][_]data.json"
    file_name_list = glob.glob(file_pattern)

    datasets = []
    for index, file_name in enumerate(reversed(file_name_list)):

        snapshot_date = parse_date(file_name)
        dataset_list = load_file(file_name)
        dataset_list = support_old_schema(dataset)

        key_list      = get_key_list(dataset_list)
        agency_counts = get_agency_counts(key_list)

        print snapshot_date+": "+str(agency_counts)+"\n"

        if index > 0: break  # Don't run all for debugging

            
main()

2015-12-01: {'009:25': 44, '009:15': 19, '009:92': 7, '009:10': 151, '009:75': 6, '009:20': 243, '009:30': 11, '009:17': 1, '009:70': 81, '009:00': 679, '009:38': 551, '009:33': 14}

2015-11-30: {'009:25': 44, '009:15': 19, '009:92': 7, '009:10': 151, '009:75': 6, '009:20': 243, '009:30': 11, '009:17': 1, '009:70': 81, '009:00': 679, '009:38': 551, '009:33': 14}

